In [1]:
%reload_ext autoreload
%autoreload 2

In [59]:
import os.path as op
import os
import hcp
import glob
import shutil
from scipy.io import loadmat, savemat

In [3]:
subject = '105923'
base_data_path = op.expanduser('/Users/dengemann/mne-hcp-data')
testing_data_path = op.join(base_data_path, 'mne-hcp-testing')
hcp_data_path = op.join(base_data_path, 'HCP')
data_types = ['task_working_memory', 'task_motor', 'task_story_math', 'rest',
              'noise_subject', 'noise_empty_room']
test_zip = op.join(base_data_path, 'hcp105923')

# copy over edited raw files

In [4]:
def edit(fname):
    fname = (fname.replace(test_zip, testing_data_path)
                  .replace('unprocd', '%s/unprocessed/MEG' % subject)
                  .replace(',ee', ''))
    return fname

In [5]:
fnames = list()
fnames.extend(glob.glob(test_zip + "/*/*/*/config"))
fnames.extend(glob.glob(test_zip + "/*/*/*/*,ee"))
fnames_to = [edit(f) for f in fnames]
new_paths = {op.dirname(f) for f in fnames_to}
for path in new_paths:
    if not op.exists(path):
        os.makedirs(path)

In [6]:
for old_name, new_name in zip(fnames, fnames_to):
    shutil.copy(old_name, new_name)

# copy over files from download 

In [7]:
files_from_subject = hcp.file_mapping.get_s3_keys_meg(
    subject=subject, data_types=data_types,
    outputs=('epochs', 'ica', 'bads', 'trial_info'), run_inds=list(range(3)),
    hcp_path_bucket=hcp_data_path)
files_from_subject.extend(glob.glob(hcp_data_path + '/%s/T1w/*/*/*' % subject))
files_from_subject.extend(hcp.file_mapping.get_s3_keys_anatomy(
    subject=subject,
    freesurfer_outputs=(),
    hcp_path_bucket=hcp_data_path))

In [8]:
files_to_testing = [ff.replace(hcp_data_path, testing_data_path) for ff in files_from_subject]

In [65]:
new_paths = {op.dirname(f) for f in files_to_testing}
for path in new_paths:
    if not op.exists(path):
        os.makedirs(path)

In [67]:
for old_name, new_name in zip(files_from_subject,
                              files_to_testing):
    shutil.copy(old_name, new_name)

# edit epochs files

In [39]:
epochs_files = list()
for data_type in data_types:
    if 'noise' in data_type:
        continue
    n_runs = len(hcp.io.file_mapping.file_mapping.run_map[data_type])
    for run_index in range(n_runs): 
        epochs_files.extend(
            [hcp.file_mapping.get_file_paths(
                subject=subject, hcp_path=testing_data_path,
                output='epochs', data_type=data_type, onset=onset, run_index=run_index)[0]
             for onset in ('stim', 'resp')])

In [43]:
this_epochs_files = [ff for ff in
                     glob.glob(op.join(testing_data_path, '105923/MEG/*/*megpreproc/*mat'))
                     if ff in epochs_files]

In [378]:
def decim_hcp_mat_epochs(fname, decim=100):
    mat = loadmat(fname, squeeze_me=False)
    data = mat['data']
    for ii, this_times in enumerate(data['time'][0][0][0]):
        data['time'][0][0][0][ii] = this_times[:, ::decim]
    for ii, this_trial in enumerate(data['trial'][0][0][0]):
        data['trial'][0][0][0][ii] = this_trial[:, ::decim]
    data['fsample'][0][0][0][0] /= float(decim)
    mat['data'] = data
    savemat(fname, mat)


In [379]:
for fname in this_epochs_files:
    decim_hcp_mat_epochs(fname)

# take care of ICA

In [380]:
ica_fnames = glob.glob(op.join(testing_data_path, '105923/MEG/*/*ica*/*mat'))

def decim_hcp_mat_ica(fname):
    mat = loadmat(fname, squeeze_me=False)
    data = mat['comp_class']
    data['trial'] = 0
    mat['comp_class'] = data
    mat['options'] = []
    savemat(fname, mat)
    
for fname in ica_fnames:
    decim_hcp_mat_ica(fname)

# Weird bug with mne EpochsArray times constructor

In [68]:
fname = this_epochs_files[-3]

In [289]:
mat = loadmat(fname, squeeze_me=False)

In [290]:
data = mat['data']

In [291]:
decim = 1

In [292]:
times = data['time'][0][0][0][0][:, ::decim]

In [331]:
for ii, this_trial in enumerate(data['trial'][0][0][0]):
    data['trial'][0][0][0][ii] = this_trial[:, ::decim]

In [354]:
shape = data['trial'][0][0][0].shape + data['trial'][0][0][0][0].shape
shape

(195, 245, 2798)

In [333]:
tmin = times.min()

In [369]:
sfreq = data['fsample'][0][0][0][0] / float(decim)

In [370]:
tmax = (shape[2] - 1) / sfreq + tmin

In [371]:
tmax == times.max()

False

In [372]:
tmin = -1.5

In [373]:
start_idx = int(round(tmin * sfreq))

In [374]:
rec_times = np.arange(start_idx,
                      int(round(tmax * sfreq)) + 1) / float(sfreq)

In [375]:
np.allclose(times, rec_times, atol=0.000001)

False

In [376]:
np.allclose(times, np.linspace(tmin, tmax, shape[2]), atol=0.000001)

False

In [377]:
rec_times

array([-1.50011543, -1.49814936, -1.49618328, ...,  3.99506496,
        3.99703103,  3.99899711])

In [393]:
np.allclose??